In [25]:
from hana_ml import DataFrame, ConnectionContext
from hana_ml.dataframe import create_dataframe_from_pandas
from hana_ml.algorithms.apl.gradient_boosting_classification import GradientBoostingBinaryClassifier
from hana_ml.algorithms.pal.partition import train_test_val_split
from hana_ml.visualizers.unified_report import UnifiedReport

from shap._explanation import Explanation
from shap.plots import force

In [2]:
hana_address = '8c7eed2e-f460-4a82-abfa-e3ede36923d7.hna1.prod-eu10.hanacloud.ondemand.com' 
hana_port = 443
hana_user = 'MLCU_008' 
hana_password = 'Welcome22' 
hana_encrypt = 'true'

# Instantiate connection object
conn = ConnectionContext(address = hana_address,
                                   port = 443, 
                                   user = hana_user, 
                                   password = hana_password, 
                                   encrypt = hana_encrypt,
                                   sslValidateCertificate = 'false' 
                                  )


At this point we are assuming that table EMP_CHURN contains the data from the CSV file. This step has been done in the "Data Upload" notebook.

We will split up the data into a train and test set. We do not need a validation set as the APL will already split the train set into a train and validation set internally. The test set here is simply used as a hold-out set which will not get used for training the algorithm.

In [3]:
emp_churn_all = DataFrame(conn, 'select * from EMP_CHURN')

In [4]:
train, test, valid = train_test_val_split(emp_churn_all, testing_percentage=0.2, validation_size=0)

Now store the train and test datasets into their own tables:

In [5]:
train.save('EMP_CHURN_TRAIN', force=True)
test.save('EMP_CHURN_TEST', force=True)

The data from the train table will be used to fit a classifier:

In [6]:
model = GradientBoostingBinaryClassifier(variable_auto_selection = True)

model.set_params(
    extra_applyout_settings={
        'APL/ApplyExtraMode': 'Advanced Apply Settings',
        'APL/ApplyPredictedValue': 'true',
        'APL/ApplyProbability': 'true',       
        'APL/ApplyDecision': 'true',   
        'APL/ApplyContribution': 'all' 
    })

model.fit(train, label='FLIGHT_RISK', key='EMPLOYEE_ID')

Now apply the trained model on the hold-out dataset and view a few examples:

In [7]:
apply_out = model.predict(test)
pdf_apply_out = apply_out.collect()


print(pdf_apply_out.head(100))

    EMPLOYEE_ID TRUE_LABEL PREDICTED  gb_score_FLIGHT_RISK  \
0         10156         No        No             -4.033816   
1         10158         No        No             -5.649900   
2         10202        Yes        No             -2.510393   
3         10255         No        No             -2.641142   
4         10265         No        No             -4.911518   
..          ...        ...       ...                   ...   
95        15774         No        No             -1.619826   
96        15787        Yes        No             -2.149041   
97        15790        Yes        No             -1.334531   
98        15791         No        No             -2.067872   
99        15794         No        No             -2.509468   

    gb_proba_FLIGHT_RISK  gb_contrib_AGE  gb_contrib_RISK_OF_LOSS  \
0               0.017399        0.159839                -0.181190   
1               0.003506       -0.169122                -0.130033   
2               0.075133        0.071276        

We will use the AUC metric for retrieving the classifier's performance. This will be in the interval [0..1] where we would like it to be close to 1.

In [8]:
model.get_performance_metrics()['AUC']

0.9021

Let's see an overview of the feature importances. This lists the variables from the dataset contributing the most towards the flight risk from top to bottom.

The numbers display their share of importance, summing up to 1.

In [9]:
UnifiedReport(model).build().display()

In order to review the unified classification model report better, you need to adjust the size of the left area or hide the left area temporarily!


In [10]:
model.get_feature_importances()

{'ExactSHAP': OrderedDict([('FUNCTIONALAREACHANGETYPE', 0.2362622767686844),
              ('PROMOTION_WITHIN_LAST_3_YEARS', 0.14471279084682465),
              ('TIMEINPREVPOSITIONMONTH', 0.10079845041036606),
              ('EMPLOYMENT_TYPE_2', 0.08776219934225082),
              ('JOBLEVELCHANGETYPE', 0.049197081476449966),
              ('AGE', 0.03487208113074303),
              ('TENURE_MONTHS', 0.03410685062408447),
              ('CURCOUNTRYLON', 0.030964959412813187),
              ('RISK_OF_LOSS', 0.026749497279524803),
              ('SALARY', 0.026116052642464638),
              ('GENDER', 0.025434203445911407),
              ('PREVCOUNTRYLON', 0.024762123823165894),
              ('CHANGE_IN_PERFORMANCE_RATING', 0.022422339767217636),
              ('IMPACT_OF_LOSS', 0.01979353092610836),
              ('PREVIOUS_PERFORMANCE_RATING', 0.019396748393774033),
              ('PREVCOUNTRYLAT', 0.018959496170282364),
              ('FUTURE_LEADER', 0.01774841547012329),
        

It appears that the field FUNCTIONALAREACHANGETYPE is the most important towards predicting the target. Lets see what this field contains:

In [11]:
emp_churn_all.agg([('count', 'FUNCTIONALAREACHANGETYPE', 'COUNT')], group_by='FUNCTIONALAREACHANGETYPE').collect()

,FUNCTIONALAREACHANGETYPE,COUNT
0,No change,5220
1,Cross-Functional Move,5995
2,External Hire,1301
3,Intra-Functional Move,6599


In [12]:
dict = { 'TRUE_LABEL': 'Target Actual',
        'PREDICTED': 'Target Predicted',
        'gb_score_FLIGHT_RISK': 'Score', 
        'gb_proba_FLIGHT_RISK': 'Probability',        
        'gb_contrib_constant_bias': 'Shap Baseline' }

pdf_apply_out.rename(columns=dict, inplace=True)    
pdf_apply_out

,EMPLOYEE_ID,Target Actual,Target Predicted,Score,Probability,gb_contrib_AGE,gb_contrib_RISK_OF_LOSS,gb_contrib_IMPACT_OF_LOSS,gb_contrib_FUTURE_LEADER,gb_contrib_GENDER,...,gb_contrib_CURRENT_FUNCTIONAL_AREA,gb_contrib_CURRENT_PERFORMANCE_RATING,gb_contrib_CURRENT_COUNTRY,gb_contrib_CURCOUNTRYLAT,gb_contrib_CURCOUNTRYLON,gb_contrib_PROMOTION_WITHIN_LAST_3_YEARS,gb_contrib_CHANGE_IN_PERFORMANCE_RATING,gb_contrib_FUNCTIONALAREACHANGETYPE,gb_contrib_JOBLEVELCHANGETYPE,Shap Baseline
0,10156,No,No,-4.033816,0.017399,0.159839,-0.181190,-0.234545,-0.103532,0.166820,...,0.009315,-0.146584,-0.173253,-0.088201,-0.213927,-1.347492,-0.073196,0.408891,-0.002687,-2.159733
1,10158,No,No,-5.649900,0.003506,-0.169122,-0.130033,-0.005680,-0.154848,-0.133955,...,-0.011181,-0.003616,-0.096269,-0.075363,-0.180102,-1.860587,-0.064217,0.290383,0.079192,-2.159733
2,10202,Yes,No,-2.510393,0.075133,0.071276,0.250221,-0.035902,-0.063465,-0.280444,...,0.048166,-0.013878,-0.033154,-0.084552,-0.145366,0.234241,-0.112750,0.282719,0.141055,-2.159733
3,10255,No,No,-2.641142,0.066537,0.213506,0.131356,0.014168,0.352461,-0.135610,...,-0.014957,-0.002150,-0.099268,-0.072370,-0.152506,-0.970940,-0.047128,0.402152,0.056643,-2.159733
4,10265,No,No,-4.911518,0.007308,-0.063499,-0.131113,0.043053,-0.124944,-0.164890,...,-0.040550,-0.020568,-0.096606,-0.074271,-0.182599,-1.695886,-0.060285,0.321169,0.080140,-2.159733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3818,28592,No,Yes,0.688680,0.665673,-0.237073,0.036236,-0.100062,-0.042473,0.108156,...,0.072958,0.040776,0.177975,0.066004,0.309625,0.363700,0.735554,0.262058,-0.000048,-2.159733
3819,28665,No,No,-1.097617,0.250187,-0.242472,0.008810,-0.078376,-0.042075,0.119641,...,0.064940,0.000683,-0.151019,0.049010,0.187974,0.352693,-0.079017,0.270599,-0.145161,-2.159733
3820,28695,No,Yes,-0.803629,0.309250,-0.229837,0.020095,-0.039008,-0.064639,0.113857,...,0.064249,-0.060864,-0.015536,0.065405,0.147478,0.321191,-0.145011,0.268369,-0.012784,-2.159733
3821,28742,No,Yes,0.435786,0.607254,-0.222074,0.119942,-0.057814,-0.046956,0.086573,...,0.053550,0.001759,0.053106,0.053001,-0.265395,0.329075,0.609578,0.262250,-0.038989,-2.159733


In [13]:
pdf_apply_out.columns

#cols = [hdr.replace('gb_contrib_', 'Shap ') for hdr in pdf_apply_out.columns]     
#cols

pdf_apply_out.columns = [hdr.replace('gb_contrib_', 'Shap ') for hdr in pdf_apply_out.columns]     

col_list = [col for col in pdf_apply_out.columns if col.startswith('Shap')]
pdf_apply_out['Total Shap'] = pdf_apply_out[col_list].sum(axis=1)

pdf_apply_out

,EMPLOYEE_ID,Target Actual,Target Predicted,Score,Probability,Shap AGE,Shap RISK_OF_LOSS,Shap IMPACT_OF_LOSS,Shap FUTURE_LEADER,Shap GENDER,...,Shap CURRENT_PERFORMANCE_RATING,Shap CURRENT_COUNTRY,Shap CURCOUNTRYLAT,Shap CURCOUNTRYLON,Shap PROMOTION_WITHIN_LAST_3_YEARS,Shap CHANGE_IN_PERFORMANCE_RATING,Shap FUNCTIONALAREACHANGETYPE,Shap JOBLEVELCHANGETYPE,Shap Baseline,Total Shap
0,10156,No,No,-4.033816,0.017399,0.159839,-0.181190,-0.234545,-0.103532,0.166820,...,-0.146584,-0.173253,-0.088201,-0.213927,-1.347492,-0.073196,0.408891,-0.002687,-2.159733,-4.033816
1,10158,No,No,-5.649900,0.003506,-0.169122,-0.130033,-0.005680,-0.154848,-0.133955,...,-0.003616,-0.096269,-0.075363,-0.180102,-1.860587,-0.064217,0.290383,0.079192,-2.159733,-5.649901
2,10202,Yes,No,-2.510393,0.075133,0.071276,0.250221,-0.035902,-0.063465,-0.280444,...,-0.013878,-0.033154,-0.084552,-0.145366,0.234241,-0.112750,0.282719,0.141055,-2.159733,-2.510392
3,10255,No,No,-2.641142,0.066537,0.213506,0.131356,0.014168,0.352461,-0.135610,...,-0.002150,-0.099268,-0.072370,-0.152506,-0.970940,-0.047128,0.402152,0.056643,-2.159733,-2.641143
4,10265,No,No,-4.911518,0.007308,-0.063499,-0.131113,0.043053,-0.124944,-0.164890,...,-0.020568,-0.096606,-0.074271,-0.182599,-1.695886,-0.060285,0.321169,0.080140,-2.159733,-4.911518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3818,28592,No,Yes,0.688680,0.665673,-0.237073,0.036236,-0.100062,-0.042473,0.108156,...,0.040776,0.177975,0.066004,0.309625,0.363700,0.735554,0.262058,-0.000048,-2.159733,0.688679
3819,28665,No,No,-1.097617,0.250187,-0.242472,0.008810,-0.078376,-0.042075,0.119641,...,0.000683,-0.151019,0.049010,0.187974,0.352693,-0.079017,0.270599,-0.145161,-2.159733,-1.097616
3820,28695,No,Yes,-0.803629,0.309250,-0.229837,0.020095,-0.039008,-0.064639,0.113857,...,-0.060864,-0.015536,0.065405,0.147478,0.321191,-0.145011,0.268369,-0.012784,-2.159733,-0.803629
3821,28742,No,Yes,0.435786,0.607254,-0.222074,0.119942,-0.057814,-0.046956,0.086573,...,0.001759,0.053106,0.053001,-0.265395,0.329075,0.609578,0.262250,-0.038989,-2.159733,0.435786


In [14]:
shap_columns = col_list[:-5]
predictors_names = [c[5:] for c in shap_columns if not c.startswith('DATE')]
shap_values = pdf_apply_out[shap_columns].values




In [15]:
pdf_apply_out

,EMPLOYEE_ID,Target Actual,Target Predicted,Score,Probability,Shap AGE,Shap RISK_OF_LOSS,Shap IMPACT_OF_LOSS,Shap FUTURE_LEADER,Shap GENDER,...,Shap CURRENT_PERFORMANCE_RATING,Shap CURRENT_COUNTRY,Shap CURCOUNTRYLAT,Shap CURCOUNTRYLON,Shap PROMOTION_WITHIN_LAST_3_YEARS,Shap CHANGE_IN_PERFORMANCE_RATING,Shap FUNCTIONALAREACHANGETYPE,Shap JOBLEVELCHANGETYPE,Shap Baseline,Total Shap
0,10156,No,No,-4.033816,0.017399,0.159839,-0.181190,-0.234545,-0.103532,0.166820,...,-0.146584,-0.173253,-0.088201,-0.213927,-1.347492,-0.073196,0.408891,-0.002687,-2.159733,-4.033816
1,10158,No,No,-5.649900,0.003506,-0.169122,-0.130033,-0.005680,-0.154848,-0.133955,...,-0.003616,-0.096269,-0.075363,-0.180102,-1.860587,-0.064217,0.290383,0.079192,-2.159733,-5.649901
2,10202,Yes,No,-2.510393,0.075133,0.071276,0.250221,-0.035902,-0.063465,-0.280444,...,-0.013878,-0.033154,-0.084552,-0.145366,0.234241,-0.112750,0.282719,0.141055,-2.159733,-2.510392
3,10255,No,No,-2.641142,0.066537,0.213506,0.131356,0.014168,0.352461,-0.135610,...,-0.002150,-0.099268,-0.072370,-0.152506,-0.970940,-0.047128,0.402152,0.056643,-2.159733,-2.641143
4,10265,No,No,-4.911518,0.007308,-0.063499,-0.131113,0.043053,-0.124944,-0.164890,...,-0.020568,-0.096606,-0.074271,-0.182599,-1.695886,-0.060285,0.321169,0.080140,-2.159733,-4.911518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3818,28592,No,Yes,0.688680,0.665673,-0.237073,0.036236,-0.100062,-0.042473,0.108156,...,0.040776,0.177975,0.066004,0.309625,0.363700,0.735554,0.262058,-0.000048,-2.159733,0.688679
3819,28665,No,No,-1.097617,0.250187,-0.242472,0.008810,-0.078376,-0.042075,0.119641,...,0.000683,-0.151019,0.049010,0.187974,0.352693,-0.079017,0.270599,-0.145161,-2.159733,-1.097616
3820,28695,No,Yes,-0.803629,0.309250,-0.229837,0.020095,-0.039008,-0.064639,0.113857,...,-0.060864,-0.015536,0.065405,0.147478,0.321191,-0.145011,0.268369,-0.012784,-2.159733,-0.803629
3821,28742,No,Yes,0.435786,0.607254,-0.222074,0.119942,-0.057814,-0.046956,0.086573,...,0.001759,0.053106,0.053001,-0.265395,0.329075,0.609578,0.262250,-0.038989,-2.159733,0.435786


In [16]:

actual_values = test.collect()[predictors_names].values
baseline = pdf_apply_out['Shap Baseline']
xpl = Explanation(shap_values, base_values=baseline, data=actual_values, feature_names=predictors_names)

In [29]:
idx = pdf_apply_out[pdf_apply_out['EMPLOYEE_ID'] == '10156'].index.values.astype(int)[0]
force(xpl[0].base_values, xpl[idx].values, features=xpl[idx].data, feature_names=predictors_names, matplotlib=True)

IndexError: index 0 is out of bounds for axis 0 with size 0

This tells that field FUNCTIONALAREACHANGETYPE incidates whether the employee has recently been allowed an Intra-functional or Cross-functional move or whether there has been no change (we will ignore the External hires as the company has no influence on their churn).

Now let's examing the employees for which the model predicts they will be churning:

In [17]:
emp_flightrisk = apply_out.filter('PREDICTED = \'Yes\'')
num_flightrisk = emp_flightrisk.describe('EMPLOYEE_ID').collect()['count'].values[0]
print('Number of employees in test set with positive flight risk: {}'.format(num_flightrisk))

Number of employees in test set with positive flight risk: 470


The above shows the number of employees in the test set with a risk of churning according to the statistical model.

Now we will take this table of employees with a flight risk and join it with the original table as loaded from the input CSV. This will list all details of the employees with a churn risk.

In [18]:
emp_flightrisk_new = emp_flightrisk.alias('L').join(emp_churn_all.alias('R'), 'L.EMPLOYEE_ID = R.EMPLOYEE_ID', select=[
    ('L.EMPLOYEE_ID', 'EMPLOYEE_ID'),
    'AGE',
    'AGE_GROUP10',
    'AGE_GROUP5',
    'GENERATION',
    'CRITICAL_JOB_ROLE',
    'RISK_OF_LOSS',
    'IMPACT_OF_LOSS',
    'FUTURE_LEADER',
    'GENDER',
    'MGR_EMP',
    'MINORITY',
    'TENURE_MONTHS',
    'TENURE_INTERVAL_YEARS',
    'TENURE_INTERVALL_DESC',
    'SALARY',
    'EMPLOYMENT_TYPE',
    'EMPLOYMENT_TYPE_2',
    'HIGH_POTENTIAL',
    'PREVIOUS_FUNCTIONAL_AREA',
    'PREVIOUS_JOB_LEVEL',
    'PREVIOUS_CAREER_PATH',
    'PREVIOUS_PERFORMANCE_RATING',
    'PREVIOUS_COUNTRY',
    'PREVCOUNTRYLAT',
    'PREVCOUNTRYLON',
    'PREVIOUS_REGION',
    'TIMEINPREVPOSITIONMONTH',
    'CURRENT_FUNCTIONAL_AREA',
    'CURRENT_JOB_LEVEL',
    'CURRENT_CAREER_PATH',
    'CURRENT_PERFORMANCE_RATING',
    'CURRENT_REGION',
    'CURRENT_COUNTRY',
    'CURCOUNTRYLAT',
    'CURCOUNTRYLON',
    'PROMOTION_WITHIN_LAST_3_YEARS',
    'CHANGED_POSITION_WITHIN_LAST_2_YEARS',
    'CHANGE_IN_PERFORMANCE_RATING',
    'FUNCTIONALAREACHANGETYPE',
    'JOBLEVELCHANGETYPE',
    'HEADS',
    #'FLIGHT_RISK'
])

pdf_emp_flightrisk = emp_flightrisk_new.collect()

Now let's see what the functional area change types are for those employees with a flight risk:

In [19]:
pdf_emp_flightrisk['FUNCTIONALAREACHANGETYPE'].value_counts()

Intra-Functional Move    262
No change                107
Cross-Functional Move     94
External Hire              7
Name: FUNCTIONALAREACHANGETYPE, dtype: int64

To check the effect of the functional area change type for these employees with a churn risk, their functional area change type will be set to 'Cross-functional move':

In [20]:
#pdf_emp_flightrisk['FUNCTIONALAREACHANGETYPE'] = 'Intra-Functional Move'
pdf_emp_flightrisk['FUNCTIONALAREACHANGETYPE'] = 'Cross-Functional Move'


pdf_emp_flightrisk


,EMPLOYEE_ID,AGE,AGE_GROUP10,AGE_GROUP5,GENERATION,CRITICAL_JOB_ROLE,RISK_OF_LOSS,IMPACT_OF_LOSS,FUTURE_LEADER,GENDER,...,CURRENT_REGION,CURRENT_COUNTRY,CURCOUNTRYLAT,CURCOUNTRYLON,PROMOTION_WITHIN_LAST_3_YEARS,CHANGED_POSITION_WITHIN_LAST_2_YEARS,CHANGE_IN_PERFORMANCE_RATING,FUNCTIONALAREACHANGETYPE,JOBLEVELCHANGETYPE,HEADS
0,10362,48,(45-55],(45-50],Generation X,Non-Critical,Medium,High,Future Leader,Female,...,EMEA,Germany,51.083420,10.423447,No Promotion,No Change,0 - Not available,Cross-Functional Move,No change,1
1,13297,33,(25-35],(30-35],Generation Y,Critical,Low,Medium,Future Leader,Male,...,EMEA,Germany,51.083420,10.423447,No Promotion,No Change,0 - Not available,Cross-Functional Move,No change,1
2,13404,42,(35-45],(40-45],Generation X,Critical,Low,Medium,No Future Leader,Male,...,EMEA,Germany,51.083420,10.423447,No Promotion,No Change,0 - Not available,Cross-Functional Move,External Hire,1
3,13668,39,(35-45],(35-40],Generation X,Critical,Low,Medium,No Future Leader,Male,...,EMEA,Germany,51.083420,10.423447,No Promotion,No Change,0 - Not available,Cross-Functional Move,External Hire,1
4,13684,39,(35-45],(35-40],Generation X,Critical,Low,Medium,No Future Leader,Male,...,EMEA,Germany,51.083420,10.423447,No Promotion,No Change,0 - Not available,Cross-Functional Move,No change,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,14800,50,(45-55],(45-50],Generation X,Non-Critical,Low,High,Future Leader,Male,...,Americas,Canada,61.066692,-107.991707,No Promotion,Change,1 - Increasing,Cross-Functional Move,Same Level,1
466,14856,44,(35-45],(40-45],Generation X,Critical,Low,High,No Future Leader,Female,...,Americas,USA,39.783730,-100.445882,No Promotion,No Change,0 - Not available,Cross-Functional Move,No change,1
467,15187,48,(45-55],(45-50],Generation X,Non-Critical,Medium,Medium,Future Leader,Male,...,APJ,Singapore,1.290475,103.852036,No Promotion,No Change,0 - Not available,Cross-Functional Move,No change,1
468,15292,42,(35-45],(40-45],Generation X,Non-Critical,Low,Medium,Future Leader,Male,...,APJ,China,35.000074,104.999927,No Promotion,No Change,0 - Not available,Cross-Functional Move,No change,1


In [21]:
create_dataframe_from_pandas(conn, pdf_emp_flightrisk, 'CHURNING_EMPLOYEES', force=True)

100%|██████████| 1/1 [00:00<00:00,  4.52it/s]


In [22]:
emp_churning = DataFrame(conn, 'select * from CHURNING_EMPLOYEES')
apply_out_new = model.predict(emp_churning)

In [23]:
emp_flightrisk_new_pos = apply_out_new.filter('PREDICTED = \'Yes\'')
num_flightrisk_new = emp_flightrisk_new_pos.describe('EMPLOYEE_ID').collect()['count'].values[0]
num_flightrisk_delta = num_flightrisk - num_flightrisk_new
print('Number of employees in test set with positive flight risk after change in Functional Area Change Type from No change to Cross-Functional Move: {}'.format(num_flightrisk_new))

print('This is down {}, which means that {:.1f}% of employees can possibly be prevented from churning by allowing them a Cross-Functional Move'.format(num_flightrisk_delta, num_flightrisk_delta / num_flightrisk * 100))

Number of employees in test set with positive flight risk after change in Functional Area Change Type from No change to Cross-Functional Move: 165
This is down 305, which means that 64.9% of employees can possibly be prevented from churning by allowing them a Cross-Functional Move
